In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, explained_variance_score


from general_code import *

In [10]:
#Read using function
y1, y2, y3, y4, X = william.data_initialization()
X.head()

,enzyme_complex_concentration,enzyme_concentration,mu_mass,product_concentration,substrate_concentration,volume_fraction,sigma_mass_0.0,sigma_mass_0.825
0,-9.761988,-11.959213,12.1,-9.230543,-9.923690,0.2,1,0
1,-11.959213,-9.761988,12.1,-8.825078,-8.314252,0.2,1,0
2,-10.860600,-10.013302,31.9,-11.022305,-10.616837,0.2,0,1
3,-11.266065,-9.879771,12.1,-8.537396,-11.309985,0.4,1,0
4,-11.959213,-9.761988,12.1,-9.923690,-8.825078,0.2,1,0


In [25]:
Y = np.column_stack((y1, y1, y3, y4))

# Plots

Seems useless, nothing to see

# Data engineering

### Interactions w/o categories

### Interaction w/ categories
works better. Best performance: _Degree 2 complete_

In [11]:
X = william.polynomial_data(X)

In [12]:
checkX = pd.DataFrame(X[:10,:])
checkX.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,-9.761988,-11.959213,12.1,-9.230543,-9.923690,0.2,95.296410,116.745689,-118.120055,90.108451,...,-9.923690,0.2,-0.0000,-0.000000,0.0,-0.000000,-0.000000,0.0,1.0,0.0
1,-11.959213,-9.761988,12.1,-8.825078,-8.314252,0.2,143.022765,116.745689,-144.706472,105.540983,...,-8.314252,0.2,-0.0000,-0.000000,0.0,-0.000000,-0.000000,0.0,1.0,0.0
2,-10.860600,-10.013302,31.9,-11.022305,-10.616837,0.2,117.952638,108.750475,-346.453149,119.708844,...,-0.000000,0.0,-10.8606,-10.013302,31.9,-11.022305,-10.616837,0.2,0.0,1.0
3,-11.266065,-9.879771,12.1,-8.537396,-11.309985,0.4,126.924229,111.306146,-136.319391,96.182860,...,-11.309985,0.4,-0.0000,-0.000000,0.0,-0.000000,-0.000000,0.0,1.0,0.0
4,-11.959213,-9.761988,12.1,-9.923690,-8.825078,0.2,143.022765,116.745689,-144.706472,118.679521,...,-8.825078,0.2,-0.0000,-0.000000,0.0,-0.000000,-0.000000,0.0,1.0,0.0


# Regressions

## Ridge

### Train test splitting

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=3, test_size=0.4)
print(X_train.shape, X_test.shape)

(91173, 41) (60783, 41)


### Fitting

In [35]:
ridge_reg = RidgeCV(alphas=np.logspace(-5,5,40), fit_intercept=False).fit(X_train, y_train)

print('Train R^2 score : ', ridge_reg.score(X_train, y_train), '\n',
     'Test R^2 score : ', ridge_reg.score(X_test, y_test))

Train R^2 score :  0.9836161706081069 
 Test R^2 score :  0.9838746977382005


In [36]:
y_pred_tr = ridge_reg.predict(X_train)
y_pred_te = ridge_reg.predict(X_test)

print('MSE on train : ', mean_squared_error(y_train, y_pred_tr), '\n',
     'MSE on test : ', mean_squared_error(y_test, y_pred_te))


MSE on train :  0.014328700046541229 
 MSE on test :  0.014205989872999127


In [37]:
print(y_pred_tr[:10, :])

[[ 0.38280705  0.38280705 13.82155197  1.16908756]
 [ 1.92386671  1.92386671 13.29411817  2.70900729]
 [ 1.40820672  1.40820672 13.32441205  2.1952175 ]
 [ 0.14888191  0.14888191 13.94532294  0.93430249]
 [ 2.09379809  2.09379809 13.26844499  2.87938362]
 [ 1.50743619  1.50743619 13.55629321  2.29236196]
 [ 2.1238462   2.1238462  13.00256215  2.90875803]
 [ 1.5153284   1.5153284  13.56548684  2.30007723]
 [ 1.50538728  1.50538728 13.57047315  2.29032986]
 [ 0.1416807   0.1416807  13.95718768  0.92704898]]


In [11]:
print('alpha : ', ridge_reg.alpha_,'\n',
     'Coeff : ', ridge_reg.coef_)

alpha :  0.003665241237079626 
 Coeff :  [ 1.40099101e-02 -7.03980994e-02 -3.92428024e-02  5.02744229e-02
 -4.51612149e-02 -4.91458286e-02 -1.44835325e+00  1.31997844e-02
  8.10125627e-04  8.17234446e-04 -1.41009590e-02 -1.01627469e-04
  1.83142034e-04 -1.88953448e-04 -3.11658506e-03 -3.51916379e-02
 -3.52064617e-02  3.02798060e-03 -1.45337722e-04  4.10709773e-04
 -4.27640060e-04  6.83188972e-03 -2.11970139e-02 -1.80457878e-02
 -1.60080405e-03  4.07090122e-05 -2.27312648e-05  1.11969489e-01
  2.44314134e-02  2.58430079e-02 -3.49407711e-03 -1.82472524e-04
 -1.87094169e-02 -2.26375752e-02 -2.25236390e-02 -3.27078944e-03
 -1.64966589e-02 -2.59135698e-02 -2.32322581e-02 -1.23393055e+01
 -1.11338067e+00 -3.34972584e-01  1.31997844e-02  0.00000000e+00
  8.10125606e-04]


In [14]:
from sklearn import feature_selection

In [15]:
selection = feature_selection.RFE(ridge_reg).fit(X_train, y_train)

y_pred_tr = selection.predict(X_train)
y_pred_te = selection.predict(X_test)

print()
print('MSE on train : ', mean_squared_error(y_train, y_pred_tr), '\n',
     'MSE on test : ', mean_squared_error(y_test, y_pred_te))


NameError: name 'feature_selection' is not defined

## Support vector regression

### Train test splitting

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y1, random_state=0, test_size=0.4)
print(X_train.shape, X_test.shape)

(91173, 8) (60783, 8)


### Fitting

In [ ]:
svm_reg = SVR(kernel='poly', degree=2).fit(X_train,y_train)

print('Train R^2 score : ', svm_reg.score(X_train, y_train), '\n',
     'Test R^2 score : ', svm_reg.score(X_test, y_test))

In [ ]:
y_pred_tr = svm_reg.predict(X_train)
y_pred_te = svm_reg.predict(X_test)

print('MSE on train : ', mean_squared_error(y_train, y_pred_tr), '\n',
     'MSE on test : ', mean_squared_error(y_test, y_pred_te))

## Boosting

### Train test splitting

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y1, random_state=5, test_size=0.7)
print(X_train.shape, X_test.shape)

(45586, 41) (106370, 41)


### Fitting

In [31]:
import xgboost as xgb

In [32]:
param = {'max_depth': 10, 'eta': 1, 'silent': 1, 'subsample': 0.5}
# param['nthread'] = 4
param['reg_alpha'] = 0.7
# param['reg_lamda'] = 0.5
param['tree_method'] = 'auto'

In [33]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

xgbReg = xgb.train(params=param, dtrain=dtrain)

y_pred_tr = xgbReg.predict(dtrain)
y_pred_te = xgbReg.predict(dtest)

print('Train R^2 score : ', explained_variance_score(y_train, y_pred_tr), '\n',
     'Test R^2 score : ', explained_variance_score(y_test, y_pred_te))

print('MSE on train : ', mean_squared_error(y_train, y_pred_tr), '\n',
     'MSE on test : ', mean_squared_error(y_test, y_pred_te))



Train R^2 score :  0.9998421834479325 
 Test R^2 score :  0.9998264273990315
MSE on train :  0.00017694542410862896 
 MSE on test :  0.00019279191653453852


# Tests

In [4]:
df = pd.DataFrame({'2' : 3*[2],
                  '3' : 3*[3],
                  'cat' : [1,0,1]},
                 index = list(range(3)))
df = william.cast_categories(df, 'cat')

In [6]:
print(william.polynomial_data(df))

KeyError: "['enzyme_complex_concentration' 'enzyme_concentration' 'mu_mass'\n 'product_concentration' 'substrate_concentration' 'volume_fraction'] not in index"

In [28]:
cond = True
[1 if x else 0 for x in df['a']==1]

[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [32]:
cast_categories(df, 'a', cat_name='Number')

,b,Number 1,Number 2,Number 3
0,4,1,0,0
1,5,1,0,0
2,6,1,0,0
3,7,1,0,0
4,4,0,1,0
5,5,0,1,0
6,6,0,1,0
7,7,0,1,0
8,4,0,0,1
9,5,0,0,1
